In [1]:

import os, getpass, requests, time
from typing import Dict, Any, List

# 安全地注入 API Key（建议在 GCP 控制台为 Places API 启用后再粘贴）
if "GOOGLE_MAPS_API_KEY" not in os.environ:
    os.environ["GOOGLE_MAPS_API_KEY"] = getpass.getpass("请输入你的 Google Maps API Key（不可见）: ")

API_KEY = os.environ["GOOGLE_MAPS_API_KEY"]
BASE = "https://places.googleapis.com/v1"

def call_places(method: str, path: str, field_mask: str,
                json_body: Dict[str, Any] | None = None,
                params: Dict[str, Any] | None = None) -> Dict[str, Any]:
    """通用调用器：封装 X-Goog-FieldMask / X-Goog-Api-Key 头"""
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": field_mask,   # 必须！否则报错
    }
    url = f"{BASE}{path}"
    resp = requests.request(method, url, headers=headers, json=json_body, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()


请输入你的 Google Maps API Key（不可见）: ··········


#1.Nearby Search（指定点与半径找“餐厅”）

• 仅支持 POST，请求体里写范围（圆形中心 + 半径）。

• 示例字段掩码只拿到 ID、地理位置、地址、评分、价格等用于做餐厅画像的关键字段

In [13]:
# 你可以换成自己的坐标与半径；这里示例：新加坡 Orchard Road 附近
center_lat, center_lng = 1.3049, 103.8318  # Orchard 附近
radius_m = 1500.0

# 为 Nearby 准备一个最小够用的字段掩码（不要包含 nextPageToken —— 官方页未明确 Nearby 的翻页字段）
SEARCH_MASK_NEARBY = ",".join([
    "places.id",
    "places.displayName",
    "places.location",
    "places.formattedAddress",
    "places.primaryType",
    "places.types",
    "places.rating",
    "places.userRatingCount",
    "places.priceLevel",
    "places.googleMapsUri"
])

nearby_body = {
    "includedTypes": ["restaurant"],  # 也可用 includedPrimaryTypes
    "maxResultCount": 20,             # Nearby 单页最多 20
    "rankPreference": "POPULARITY",   # 或 "DISTANCE"
    "locationRestriction": {
        "circle": {
            "center": {"latitude": center_lat, "longitude": center_lng},
            "radius": radius_m
        }
    }
}

nearby = call_places("POST", "/places:searchNearby", SEARCH_MASK_NEARBY, json_body=nearby_body)
len(nearby.get("places", [])), nearby.get("places", [])[:2]  # 看下总数与前两个示例


(20,
 [{'id': 'ChIJe4wL_OsZ2jERySoDk6jlhNQ',
   'types': ['food_court',
    'restaurant',
    'food',
    'point_of_interest',
    'establishment'],
   'formattedAddress': '500 Clemenceau Ave N, Singapore 229495',
   'location': {'latitude': 1.3119888, 'longitude': 103.8395742},
   'rating': 4.2,
   'googleMapsUri': 'https://maps.google.com/?cid=15313617145150253769&g_mp=Cilnb29nbGUubWFwcy5wbGFjZXMudjEuUGxhY2VzLlNlYXJjaE5lYXJieRAAGAQgAA',
   'userRatingCount': 14394,
   'displayName': {'text': 'Newton Food Centre', 'languageCode': 'en'},
   'primaryType': 'food_court'},
  {'id': 'ChIJdY7JN4oZ2jERVg6-qPEGJ3U',
   'types': ['hotel',
    'lodging',
    'restaurant',
    'food',
    'point_of_interest',
    'establishment'],
   'formattedAddress': '1A Cuscaden Rd, Singapore 249716',
   'location': {'latitude': 1.3043259999999999, 'longitude': 103.8239282},
   'rating': 4.3,
   'googleMapsUri': 'https://maps.google.com/?cid=8441723661517196886&g_mp=Cilnb29nbGUubWFwcy5wbGFjZXMudjEuUGxhY2VzLl

#2.Text Search（关键词 + 区域偏置）

• POST 到 places:searchText，textQuery 可以写 “sushi near orchard road singapore”。

• pageSize 控制每页数量；响应里若返回 nextPageToken，用它继续翻页（放在请求体的 pageToken 字段）。


In [3]:
# Text Search 字段掩码：与 Nearby 类似，但加入 nextPageToken 以便分页
SEARCH_MASK_TEXT = ",".join([
    "places.id",
    "places.displayName",
    "places.location",
    "places.formattedAddress",
    "places.primaryType",
    "places.types",
    "places.rating",
    "places.userRatingCount",
    "places.priceLevel",
    "places.googleMapsUri",
    "nextPageToken"
])

text_body = {
    "textQuery": "sushi near orchard road singapore",
    # 用 locationBias 把结果“往某区域靠”
    "locationBias": {
        "circle": {
            "center": {"latitude": center_lat, "longitude": center_lng},
            "radius": 1200.0
        }
    },
    "pageSize": 10
}

page1 = call_places("POST", "/places:searchText", SEARCH_MASK_TEXT, json_body=text_body)
print("第一页条数:", len(page1.get("places", [])))
print("是否可翻页:", "nextPageToken" in page1)

# 若可翻页，继续拿第二页
page2 = {}
if "nextPageToken" in page1:
    text_body2 = {**text_body, "pageToken": page1["nextPageToken"]}
    # 按官方示例，翻页前稍等一小会儿更稳妥
    time.sleep(1.2)
    page2 = call_places("POST", "/places:searchText", SEARCH_MASK_TEXT, json_body=text_body2)
    print("第二页条数:", len(page2.get("places", [])))


第一页条数: 10
是否可翻页: True
第二页条数: 10


#3.Place Details（对单个餐厅补充细节）

• GET 到 /v1/places/{PLACE_ID}，同样必须带 FieldMask。
• 支持补充电话、网站、营业时间、评分/价位等（不同字段触发不同 SKU）

In [15]:
# 拿 Nearby 或 Text Search 的第一个结果做示例
any_place_id = (nearby.get("places") or page1.get("places") or page2.get("places"))[0]["id"]

DETAILS_MASK = ",".join([
    # 注意：Place Details 的字段掩码是 Place 对象顶层，不再以 "places." 起头！
    "id",
    "displayName",
    "location",
    "formattedAddress",
    "primaryType",
    "types",
    "rating",
    "userRatingCount",
    "priceLevel",
    "websiteUri",
    "nationalPhoneNumber",
    "currentOpeningHours",
    "googleMapsUri",
])

details = call_places("GET", f"/places/{any_place_id}", DETAILS_MASK)
details


{'id': 'ChIJe4wL_OsZ2jERySoDk6jlhNQ',
 'types': ['food_court',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'formattedAddress': '500 Clemenceau Ave N, Singapore 229495',
 'location': {'latitude': 1.3119888, 'longitude': 103.8395742},
 'rating': 4.2,
 'googleMapsUri': 'https://maps.google.com/?cid=15313617145150253769&g_mp=CiVnb29nbGUubWFwcy5wbGFjZXMudjEuUGxhY2VzLkdldFBsYWNlEAAYBCAA',
 'userRatingCount': 14394,
 'displayName': {'text': 'Newton Food Centre', 'languageCode': 'en'},
 'currentOpeningHours': {'openNow': True,
  'periods': [{'open': {'day': 2,
     'hour': 0,
     'minute': 0,
     'truncated': True,
     'date': {'year': 2025, 'month': 9, 'day': 9}},
    'close': {'day': 1,
     'hour': 23,
     'minute': 59,
     'truncated': True,
     'date': {'year': 2025, 'month': 9, 'day': 15}}}],
  'weekdayDescriptions': ['Monday: Open 24 hours',
   'Tuesday: Open 24 hours',
   'Wednesday: Open 24 hours',
   'Thursday: Open 24 hours',
   'Friday: Open 24 hours

#4.把结果整理成“餐厅画像”结构

In [8]:
import pandas as pd

def to_profile(p: Dict[str, Any]) -> Dict[str, Any]:
    lat = p.get("location", {}).get("latLng", {}).get("latitude")
    lng = p.get("location", {}).get("latLng", {}).get("longitude")
    return {
        "id": p.get("id"),
        "name": p.get("displayName", {}).get("text"),
        "lat": lat, "lng": lng,
        "address": p.get("formattedAddress"),
        "googleMapsUrl": p.get("googleMapsUri"),
        "rating": p.get("rating"),
        "ratingCount": p.get("userRatingCount"),
        "priceLevel": p.get("priceLevel"),
        "primaryType": p.get("primaryType"),
        "types": ",".join(p.get("types", []))
    }

# 合并 Nearby + Text Search 的前若干结果，做个 DataFrame 方便查看/后处理
merged_places: List[Dict[str, Any]] = []
for blk in [nearby.get("places", []), page1.get("places", []), page2.get("places", [])]:
    for p in blk:
        merged_places.append(to_profile(p))

df = pd.DataFrame(merged_places).drop_duplicates(subset=["id"]).reset_index(drop=True)
df


,id,name,lat,lng,address,googleMapsUrl,rating,ratingCount,priceLevel,primaryType,types
0,ChIJe4wL_OsZ2jERySoDk6jlhNQ,Newton Food Centre,None,None,"500 Clemenceau Ave N, Singapore 229495",https://maps.google.com/?cid=15313617145150253...,4.2,14394,None,food_court,"food_court,restaurant,point_of_interest,food,e..."
1,ChIJdY7JN4oZ2jERVg6-qPEGJ3U,JEN Singapore Tanglin by Shangri-La,None,None,"1A Cuscaden Rd, Singapore 249716",https://maps.google.com/?cid=84417236615171968...,4.3,5873,None,hotel,"hotel,lodging,restaurant,point_of_interest,foo..."
2,ChIJRe0Wq5cZ2jER5qoN9Q0znq8,Concorde Hotel Singapore,None,None,"100 Orchard Rd, Singapore 238840",https://maps.google.com/?cid=12654608137996577...,4.2,4002,None,hotel,"hotel,lodging,wedding_venue,event_venue,restau..."
3,ChIJS2f6Z40Z2jERLavQ4p6-R5s,Royal Plaza on Scotts Singapore,None,None,"25 Scotts Rd, Singapore 228220",https://maps.google.com/?cid=11189121388844722...,4.4,3386,None,hotel,"hotel,buffet_restaurant,lodging,restaurant,poi..."
4,ChIJq51Ft5YZ2jERd4ctVijFKeA,Song Fa Bak Kut Teh The Centrepoint 松發肉骨茶 先得坊,None,None,"176 Orchard Rd, #02-29/30, Singapore 238843",https://maps.google.com/?cid=16152658315576182...,4.5,1770,PRICE_LEVEL_MODERATE,restaurant,"restaurant,point_of_interest,food,establishment"
5,ChIJ_c6vFoMZ2jERBWATXfBQ7Aw,Zion Riverside Food Centre,None,None,"70 Zion Rd, Singapore 247792",https://maps.google.com/?cid=93120821624277402...,4.2,3830,PRICE_LEVEL_INEXPENSIVE,restaurant,"restaurant,point_of_interest,food,establishment"
6,ChIJn7jj_5kZ2jERdhZzwXgi41Q,Da Shi Jia Big Prawn Mee,None,None,"89 Killiney Rd, Singapore 239534",https://maps.google.com/?cid=61167706209362305...,4.1,1280,PRICE_LEVEL_MODERATE,restaurant,"restaurant,point_of_interest,food,establishment"
7,ChIJuTpnZVQZ2jEReDI4VP2u_w0,MERCI MARCEL ORCHARD,None,None,"390 Orchard Road, 01-03 Palais Renaissance, 39...",https://maps.google.com/?cid=10087172446172084...,4.7,3538,PRICE_LEVEL_MODERATE,french_restaurant,"french_restaurant,restaurant,point_of_interest..."
8,ChIJpfgvhowZ2jERbVbSD5w0-FY,Hard Rock Cafe,None,None,"50 Cuscaden Rd, #02-01 Hpl House, Singapore 24...",https://maps.google.com/?cid=62668167263710306...,4.2,2800,PRICE_LEVEL_EXPENSIVE,american_restaurant,"american_restaurant,bar,event_venue,clothing_s..."
9,ChIJDfcsE5EZ2jERrC6v6Okmdhc,Chatterbox,None,None,"333 Orchard Rd, #05-03 Hilton, Singapore 238867",https://maps.google.com/?cid=16905814961972302...,4.1,1477,PRICE_LEVEL_EXPENSIVE,restaurant,"asian_restaurant,restaurant,point_of_interest,..."
